In [5]:
!pip install -q kfp==0.2.5 --upgrade --user
!

You should consider upgrading via the 'pip install --upgrade pip' command.


In [ ]:
from IPython.core.display import HTML
HTML("<script>Jupyter.notebook.kernel.restart()</script>")

In [1]:
!which dsl-compile

/usr/local/bin/dsl-compile


In [103]:
MODEL_PATH = '/tmp'

In [114]:
import kfp
from kfp import dsl

def preprocess(path):
    return dsl.ContainerOp(
        name = 'preprocess_data',
        image = 'us.gcr.io/manceps-labs/preprocess:v0.1.0',
        command = 'python3',
        arguments = ['--path {}'.format(path)],
        file_outputs = {
            'training_data': '/tmp/train_data_0.tfrecord',
            'test_data': '/tmp/test_data_0.tfrecord',
            'prediction_data': '/tmp/pred_data_0.tfrecord',
            'test_targets': '/tmp/test_target_stats.csv'
        }
    )

def train(training_data, test_data):
    return dsl.ContainerOp(
        name = 'train_model',
        image = 'us.gcr.io/manceps-labs/train:v0.1.0',
        command = ['python3'],
        arguments = ['training.py',
                     '--training-data', training_data,
                     '--test-data', test_data
                    ],
        file_outputs = {
            'model_pb': '/tmp/saved_model.pb',
            'variables': '/tmp/variables'
        }
    )

def predict(model_path, prediction_data, test_targets):
    return dsl.ContainerOp(
        name = 'predict_model',
        image = 'us.gcr.io/manceps-labs/train:v0.1.0',
        command = ['python3'],
        arguments = ['predict.py',
                     '--model-path', model_path,
                     '--pred-data', prediction_data,
                     '--targets', test_targets
                    ]
        )
                    


In [115]:
@dsl.pipeline(
    name = 'Predict Closing Price',
    description = 'Predict the closing price of a stock with at least 20 years of historical data.'
)

def predict_stock(
    data_path = DATA_PATH
):
    preprocess_data = preprocess(data_path)
    
    train_model = train(preprocess_data.outputs['training_data'],
                        preprocess_data.outputs['test_data'],
                        
            )
            
    predict_result = predict(DATA_PATH,
                             preprocess_data.outputs['prediction_data'],
                             preprocess_data.outputs['test_targets'])
    

In [116]:
kfp.compiler.Compiler().compile(predict_stock, 'predict_stock.zip')

In [117]:
client = kfp.Client()
my_experiment = client.create_experiment(name='predict_stocks_experiment')
my_run = client.run_pipeline(my_experiment.id, 'Predict Closing Price', 
  'predict_stock.zip')

2020-05-18 22:53:25,525 WARNING Retrying (Retry(total=2, connect=None, read=None, redirect=None, status=None)) after connection broken by 'NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f81b4abbd30>: Failed to establish a new connection: [Errno -2] Name or service not known',)': /apis/v1beta1/experiments?page_token=&page_size=100&sort_by=
2020-05-18 22:53:25,525 WARNING Retrying (Retry(total=2, connect=None, read=None, redirect=None, status=None)) after connection broken by 'NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f81b4abbd30>: Failed to establish a new connection: [Errno -2] Name or service not known',)': /apis/v1beta1/experiments?page_token=&page_size=100&sort_by=
2020-05-18 22:53:25,527 WARNING Retrying (Retry(total=1, connect=None, read=None, redirect=None, status=None)) after connection broken by 'NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f81b4abb438>: Failed to establish a new connection: [Errno -2] Name o

MaxRetryError: HTTPConnectionPool(host='ml-pipeline.kubeflow.svc.cluster.local', port=8888): Max retries exceeded with url: /apis/v1beta1/experiments (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f81b4abbef0>: Failed to establish a new connection: [Errno -2] Name or service not known',))